In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
from datetime import datetime,timedelta
from sklearn import preprocessing

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 6, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False  

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Data Preprcoess

In [2]:
full_data = pd.read_csv('./input/full_data_mode_encode.csv')
full_data = full_data.drop(['Unnamed: 0'], axis=1)

full_data.shape
full_data.head()

(9934208, 68)

,id,msno,name_Album more,name_Artist more,name_Concert,name_Discover Chart,name_Discover Feature,name_Discover Genre,name_Discover New,name_Explore,...,registration_month,registration_day,genre_ids1,genre_ids2,genre_ids3,genre_ids4,genre_ids5,genre_ids6,genre_ids7,genre_ids8
0,-1,9176,0,0,0,0,0,0,0,1,...,1,2,359,0,0,0,0,0,0,0
1,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1259,0,0,0,0,0,0,0
2,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1259,0,0,0,0,0,0,0
3,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1019,0,0,0,0,0,0,0
4,-1,9176,0,0,0,0,0,0,0,1,...,1,2,1011,0,0,0,0,0,0,0


In [3]:
# for i in full_data.columns:
#     if (i != 'bd') & (i != 'registration_last_days'):
#         full_data[i] = full_data[i].astype('category')

In [3]:
train = full_data[full_data.id == -1]
test = full_data[full_data.id != -1]

ids = test.id
train = train.drop(['id'], axis=1)
test = test.drop(['id', 'target'], axis=1)

del full_data

## Train

In [4]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [5]:
x_train = train.drop(['target'], axis=1)
y_train = train.target
x_test = test

del train
del test

x_train.shape
x_test.shape
x_train.head()

(7377418, 66)

(2556790, 66)

,msno,name_Album more,name_Artist more,name_Concert,name_Discover Chart,name_Discover Feature,name_Discover Genre,name_Discover New,name_Explore,name_Local playlist more,...,registration_month,registration_day,genre_ids1,genre_ids2,genre_ids3,genre_ids4,genre_ids5,genre_ids6,genre_ids7,genre_ids8
0,9176,0,0,0,0,0,0,0,1,0,...,1,2,359,0,0,0,0,0,0,0
1,19273,0,0,0,0,0,0,0,0,1,...,5,25,1259,0,0,0,0,0,0,0
2,19273,0,0,0,0,0,0,0,0,1,...,5,25,1259,0,0,0,0,0,0,0
3,19273,0,0,0,0,0,0,0,0,1,...,5,25,1019,0,0,0,0,0,0,0
4,9176,0,0,0,0,0,0,0,1,0,...,1,2,1011,0,0,0,0,0,0,0


In [6]:
lgb_model = lgb.LGBMClassifier(
    boosting_type= 'gbdt', 
    objective = 'binary', 
    metric='auc',
#     learning_rate=0.1,
#     n_estimators=500,
#     colsample_bytree=0.75,
#     subsample=0.8,
#     reg_lambda=1.5,
#     max_depth=7,
#     min_data_in_leaf=15,
    nthread=5, 
    seed=0,
    silent=True)

In [7]:
lgb_params = {
    'learning_rate' : [0.1],
    'n_estimators' : [500],
    'max_depth': [5, 7],
#     'min_data_in_leaf':[10,15],
    'colsample_bytree' : [0.75],
    'subsample' : [0.8],
    'reg_lambda' : [1.5],
}

if __name__ == '__main__':
    lgb_grid = GridSearchCV(lgb_model, lgb_params, verbose=1, cv=3, n_jobs=-1, scoring = 'roc_auc')
    lgb_grid.fit(x_train, y_train)
    lgb_grid.best_params_
    lgb_grid.best_score_

Fitting 3 folds for each of 2 candidates, totalling 6 fits


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/multiprocessing/pool.py", line 366, in _handle_workers
    pool._maintain_pool()
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/multiprocessing/pool.py", line 240, in _maintain_pool
    self._repopulate_pool()
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/multiprocessing/pool.py", line 233, in _repopulate_pool
    w.start()
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/multiprocessing/context.py", line 267, in _Popen
    return Popen(process_obj)
  File "/usr/local/mini

TypeError: catching classes that do not inherit from BaseException is not allowed

In [10]:
lgb.plot_importance(lgb_grid.best_estimator_, max_num_features = 15)

y_test = lgb_grid.best_estimator_.predict_proba(x_test)

NameError: name 'lgb_grid' is not defined

In [7]:
lgb_model.fit(x_train, y_train)

# y_test=lgb_model.predict_proba(x_test)
y_train_pred = lgb_model.predict_proba(x_train)

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.75, learning_rate=0.1,
        max_bin=255, max_depth=7, metric='auc', min_child_samples=10,
        min_child_weight=5, min_data_in_leaf=15, min_split_gain=0,
        n_estimators=500, nthread=5, num_leaves=31, objective='binary',
        reg_alpha=0, reg_lambda=1.5, seed=0, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [12]:
sub = pd.DataFrame()
sub['id'] = ids
del ids
sub['target'] = y_test[:, 1]
sub.head()

,id,target
7377418,0,0.668502
7377419,1,0.688540
7377420,2,0.253346
7377421,3,0.229620
7377422,4,0.230352


In [13]:
sub.to_csv('./submit/lgb_4.csv', index=False, float_format = '%.6f')

In [8]:
y_pred = pd.DataFrame(y_train_pred[:, 1])
y_pred.to_csv('./submit/lgb_train.csv', index=False, float_format = '%.6f')